In [1]:
from scipy import sparse as sp
import json
import pickle
import sys
import implicit
import threadpoolctl
from tqdm import tqdm

/home/VoidWalker/Code/cs307/Bee-roll/machine_learning/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Limiting the threads
threadpoolctl.threadpool_limits(1, "blas")

In [3]:
sys.path.append("../src")
from config import NUM_PARTS, PATH_TO_ALS_MODEL
from models import ALS
from utils import MovieEncoder, make_coo_row, get_part_path

In [4]:
file = open("../models/movie.encoder", "rb")
encoder = pickle.load(file)
file.close()

In [5]:
model = implicit.als.AlternatingLeastSquares(factors=64)

In [6]:
rows = []
for index in range(NUM_PARTS):
    for data in tqdm((json.loads(part) for part in open(get_part_path(index)))):
        rows.append(make_coo_row(data["review_history"], encoder))
train_mat = sp.vstack(rows)

33300it [00:03, 10091.59it/s]
33045it [00:03, 10106.32it/s]
33011it [00:03, 9970.61it/s] 
33117it [00:03, 10065.33it/s]
33299it [00:03, 10458.26it/s]
33052it [00:03, 10224.30it/s]
32895it [00:03, 10303.21it/s]
33145it [00:03, 10088.46it/s]
32881it [00:03, 10318.16it/s]
33230it [00:03, 10124.91it/s]


In [7]:
model.fit(train_mat)

/home/VoidWalker/Code/cs307/Bee-roll/machine_learning/env/lib/python3.11/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.9629864692687988 seconds
  warnings.warn(
100%|███████████████████████████████████████████████████████████████████████| 15/15 [00:35<00:00,  2.34s/it]


In [8]:
model.user_factors = None
trained_model = ALS(model)

In [9]:
file = open(PATH_TO_ALS_MODEL, 'wb')
pickle.dump(trained_model, file)
file.close()

In [11]:
file = open(PATH_TO_ALS_MODEL, 'rb')
trained_model = pickle.load(file)
file.close()